### Preparation

✓ Open the SSH terminal and connect to the training VM <br>
Compute Engine > VM instances> training-vm > Connect. <br>
✓ in VM terminal write code ls /training <br>
✓ Download Code Repository <br>
git clone https://github.com/GoogleCloudPlatform/training-data-analyst <br>
✓ in VM terminal Set environment variable <br> 
source /training/project_env.sh <br>
✓ Prepare HBase quickstart files <br>
cd ~/training-data-analyst/courses/streaming/process/sandiego
./install_quickstart.sh <br>

### Simulate traffic sensor data into Pub/Sub
✓ in VM terminal run sensor magic code <br>
/training/sensor_magic.sh <br>
✓ upper right corner of the training-vm SSH terminal New Connection to training-vm <br>
✓ Set environment variables in second VM terminal <br>
source /training/project_env.sh <br>

### Launch Dataflow Pipeline
✓ In the second training-vm <br>
cd ~/training-data-analyst/courses/streaming/process/sandiego <br>
nano run_oncloud.sh <br>
CTRL+X # interrupt <br>
✓ Run the following script to create the Bigtable instance <br>
cd ~/training-data-analyst/courses/streaming/process/sandiego <br>
./create_cbt.sh <br>
✓ Run the Dataflow pipeline to read from PubSub and write into Cloud Bigtable <br>
cd ~/training-data-analyst/courses/streaming/process/sandiego <br>
./run_oncloud.sh $DEVSHELL_PROJECT_ID $BUCKET CurrentConditions --bigtable <br>


### Explore the pipeline , Query Bigtable data

✓ Navigation >> Dataflow >> graph >> write:cbt step, Review the Bigtable Options in Step summary <br>
✓ second training-vm <br>
cd ~/training-data-analyst/courses/streaming/process/sandiego/quickstart <br>
./quickstart.sh <br>
✓ script completes you are in HBase shell prompt <br>
scan 'current_conditions', {'LIMIT' => 2} # query to retrieve 2 rows from your Bigtable  <br>
“each row is broken into column, timestamp, value combinations” <br>
✓ This time look only at the <br> lane: speed column, limit to 10 rows, and specify rowid patterns for start 
and end rows to scan over
scan 'current_conditions', {'LIMIT' => 10, STARTROW => '15#S#1', ENDROW => '15#S#999', COLUMN 
=> 'lane:speed’} <br>
✓ Exit shell <br>
quit


### Cleanup (release sources )

1.In the second training-vm SSH terminal, run the following script to delete your Bigtable instance.
cd ~/training-data-analyst/courses/streaming/process/sandiego
./delete_cbt.sh
If prompted to confirm, enter Y.
2.On your Dataflow page in your Cloud Console, click on the pipeline job name.
3.Click Stop on the top menu bar. Select Cancel, and then click Stop Job.
4.Go back to the first SSH terminal with the publisher, and enter Ctrl+C to stop it.
5.In the BigQuery console, click on the three dots next to the demos dataset, and click Delete.
6.Type delete and then click Delete.

### theory notes:
 big table is a no sql database, it is a columnar database, it is a key value store, it is a wide column store,for large datasets (petabytes) it is a high throughput and low latency 

 how to choose between big table and big query?
 if acceptable time to insight in miliseconds then bigtable is the choice, if acceptable time to insight in seconds then big query is the choice

 in big table (cluster based service high throughput scales linearly optimized for fast store NoSQL) 100,000 queries persecond (10 nodes at 6ms latency)

 bigquery (serveless optimized for fast query SQL)
 100,000 rows per second (streaming performance)

Bigtable design idea is "simplify for speed"
Row Key is the only index

speed depend on your data and row key 

row key here is orgin#arrival

![bigtable_schema](../Media/bigtable_schema.png)

use reverse timestamps when your most common query is for the latest values 
Query : current arrival delay for flights from Atlanta
// key is ORIGIN#arrival#REVTS
string key = info.getOrigin() + "#arrival" + "#" + (Long.MAX_VALUE - ts.getMillis()); // reverse timestamp

![changeDataInBigTable](../Media/changeDataInBigTable.png)

optimizing data organization in big table for performance
* group related data together for moe efficient reads
* distribute data evenly to avoid hotspots (efficient writes)
* place identical values in same row or adjoining rows for more efficient compression

Bigtable self_improves by learning access patterns and optimizing data layout

![spotify_bigtable](../Media/spotify_bigtable.png)

![optimizing_bigtable](../Media/optimizing_bigtable.png)

![replication_bigtable](../Media/replication_bigtable.png)